<div style="text-align: center; background-color: #559cff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Lab 02 - Introduction To Data Science @ FIT-HCMUS, VNU-HCM 📌
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 4.0 - Data modelling
</div>

This section for bouns, you might need to use some basic Machine Learning algorithms to answer some questions that in cases of regression, and classification. In this scope of this subject, highly recommend you use `scikit-learn` (instead of implement once again these algorithms)

## Import

In [27]:
# YOUR CODE HERE (OPTION) 
# If you need other libraries, you can import them here
# YOUR CODE HERE (OPTION) 
# If you need other libraries, you can import them here
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
%matplotlib inline

# YOUR CODE HERE (OPTION) 
# If you need other libraries, you can import them here
import seaborn as sns

I want to predict the country classifications of the data base on : https://blogs.worldbank.org/opendata/new-world-bank-group-country-classifications-income-level-fy24#:~:text=The%20World%20Bank%E2%80%99s%20income%20classification%20aims%20to%20reflect,significantly%20over%20the%20period%20since%20the%20late%201980s.

## Data preparation

In [28]:
# YOUR CODE HERE (OPTION) 
cleaned_data = pd.read_csv('../data/processed/21120504_asia_processed.csv')

First, we need a label for each country, The label is based on GNI per capita, Atlas method (current US$)
- Low Income (<= 1.085): 1
- Lower Middle Income (1.085 - 4.225): 2
- Upper Middle Income (4.225 - 13.205): 3
- High Income (> 13.205): 4


In [41]:
BASE_URL = 'http://api.worldbank.org/v2/'

def get_another_indicator(country_code, country_name, indicator_code,indicator_name, start_year, end_year, per_page=100, format='json'):
    url = BASE_URL+'countries/' + country_code.lower() + '/indicators/' + indicator_code.lower()
    # gửi request với các tham số mặc định bằng resquests module
    params = {'format': format,
                'per_page': str(per_page),
                'date': str(start_year) + ":" + str(end_year)}
    response = requests.get(url, params=params)
    dataLst = []
    # Xác thực mã trạng thái phản hồi
    # API trả về status_code 200 ngay cả đối với các thông báo lỗi,
    # tuy nhiên, nội dung phản hồi chứa một trường có tên là "thông báo" bao gồm các chi tiết về lỗi
    # kiểm tra xem tin nhắn có xuất hiện trong phản hồi hay không
    if response.status_code == 200 and ("message" not in response.json()[0].keys()):
        # print("Successfully got data for: " + str(featureMap[indicator]))

        # danh sách các giá trị cho một đặc trưng
        indicatorVals = []

        # phản hồi là một mảng chứa hai mảng - [[{page: 1, ...}], [{year: 2018, SP.POP.TOTL: 123455}, ...]]
        # do đó chúng ta kiểm tra xem độ dài của phản hồi có > 1 hay không
        if len(response.json()) > 1:

            # nếu có, lặp lại từng đối tượng trong phản hồi
            # mỗi đối tượng cho một giá trị duy nhất cho mỗi năm
            for obj in response.json()[1]:

                # Kiểm tra giá trị rỗng
                if (obj['value'] == "") or (obj['value'] is None):
                    indicatorVals.append('None')
                else:
                    # nếu có một giá trị, hãy thêm nó vào danh sách các giá trị chỉ báo indicatorVals
                    indicatorVals.append(float(obj['value']))

            dataLst.append(indicatorVals)

        else:
            # In thông báo lỗi nếu lệnh gọi API không thành công
            print("Error in Loading the data. Status Code: " +
                    str(response.status_code))

    # Khi đã có được tất cả các đặc trưng, ta thêm các giá trị cho "Năm"
    # API trả về các giá trị indicator từ năm gần đây nhất. Do đó, chúng tôi tạo một danh sách các năm ngược lại

    dataLst.append([country_name for _ in range(
        int(end_year), int(start_year)-1, -1)])
    
    dataLst.append([year for year in range(
        int(end_year), int(start_year)-1, -1)])
    
    df = pd.DataFrame(dataLst).T
    df.columns = [indicator_name, 'Country', 'Year']
    return df

In [42]:
df_country_code = pd.read_csv('../data/external/country-codes.csv')
df_country_code.head()

,alpha-2-code,alpha-3-code,numeric,name,capital,continent,region
0,AF,AFG,4,Afghanistan,Kabul,Asia,South Asia
1,AL,ALB,8,Albania,Tirana,Europe,South East Europe
2,DZ,DZA,12,Algeria,Algiers,Africa,Northern Africa
3,AS,ASM,16,American Samoa,Pago Pago,Oceania,Pacific
4,AO,AGO,24,Angola,Luanda,Africa,Southern Africa


In [43]:
asia_country_code = df_country_code[df_country_code['continent'] == 'Asia']
asia_country_code = dict(zip(asia_country_code['alpha-2-code'].to_list(),asia_country_code['name'].to_list()))
asia_country_code

{'AF': 'Afghanistan',
 'AM': 'Armenia',
 'AZ': 'Azerbaijan',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BT': 'Bhutan',
 'BN': 'Brunei Darussalam',
 'KH': 'Cambodia',
 'CN': 'China',
 'CY': 'Cyprus',
 'GE': 'Georgia',
 'IN': 'India',
 'ID': 'Indonesia',
 'IR': 'Iran, Islamic Rep.',
 'IQ': 'Iraq',
 'IL': 'Israel',
 'JP': 'Japan',
 'JO': 'Jordan',
 'KZ': 'Kazakhstan',
 'KP': 'Korea, Dem. People’s Rep.',
 'KR': 'Korea, Rep.',
 'KW': 'Kuwait',
 'KG': 'Kyrgyz Republic',
 'LA': 'Lao PDR',
 'LB': 'Lebanon',
 'MY': 'Malaysia',
 'MV': 'Maldives',
 'MN': 'Mongolia',
 'MM': 'Myanmar',
 'NP': 'Nepal',
 'OM': 'Oman',
 'PK': 'Pakistan',
 'PH': 'Philippines',
 'QA': 'Qatar',
 'RU': 'Russian Federation',
 'SA': 'Saudi Arabia',
 'SG': 'Singapore',
 'LK': 'Sri Lanka',
 'SY': 'Syrian Arab Republic',
 'TJ': 'Tajikistan',
 'TH': 'Thailand',
 'TR': 'Turkey',
 'TM': 'Turkmenistan',
 'AE': 'United Arab Emirates',
 'UZ': 'Uzbekistan',
 'VN': 'Vietnam',
 'YE': 'Yemen, Rep.'}

In [44]:
# Create data frame with one indicator
indicator = {'NY.GNP.PCAP.CD' : 'GNI per capita, Atlas method (current US$)'}
df_gni = pd.DataFrame()
for country_code, country_name in asia_country_code.items():
    df = get_another_indicator(country_code, country_name, 'NY.GNP.PCAP.CD', indicator['NY.GNP.PCAP.CD'], 1995, 2021)
    df_gni = pd.concat([df_gni, df], ignore_index=True)
df_gni.head()

,"GNI per capita, Atlas method (current US$)",Country,Year
0,390.0,Afghanistan,2021
1,500.0,Afghanistan,2020
2,530.0,Afghanistan,2019
3,520.0,Afghanistan,2018
4,540.0,Afghanistan,2017


In [49]:
# Change dtype of column 'GNI per capita, Atlas method (current US$)' to float
df_gni['GNI per capita, Atlas method (current US$)'] = df_gni['GNI per capita, Atlas method (current US$)'].astype(float)
df_gni['GNI per capita, Atlas method (current US$)'].describe()

ValueError: could not convert string to float: 'None'

## Feature engineering

In [ ]:
# YOUR CODE HERE (OPTION) 

## Training model

In [ ]:
# YOUR CODE HERE (OPTION) 

## Evaluation

In [ ]:
# YOUR CODE HERE (OPTION) 

## Prediction

In [ ]:
# YOUR CODE HERE (OPTION) 

## Results analysis

In [ ]:
# YOUR CODE HERE (OPTION) 

## Conclusion

**Your conclusion**